In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.metrics import jaccard_similarity_score

In [2]:
drug_mat = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/8_gold_standard_drug_binary_matrix.tsv", index_col=0)

In [3]:
desc = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/2_1_descriptor_combined.tsv")

In [ ]:
len(drug_mat.columns)

In [ ]:
#2017.3.13. drug-drug_similarity 구하기
gold_drug_id = drug_mat.columns.values
df_dr_sim = pd.DataFrame()
mat_dr_sim = pd.DataFrame(columns = gold_drug_id, index = gold_drug_id)
cnt = 0
print 'start : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for i in gold_drug_id:
    desc1 = drug_mat[i]
    for j in gold_drug_id:
        desc2 = drug_mat[j]
        
        sim = jaccard_similarity_score(desc1, desc2)
        
        #dataframe형
        temp_df = pd.DataFrame(data= [[i, j , sim]], columns=['Drug_id1', 'Drug_id2', 'Similarity'])
        df_dr_sim = df_dr_sim.append(temp_df)
        # matrix형
        mat_dr_sim[i].ix[j] = sim
    cnt = cnt + 1
    if cnt % 100 == 0:
        print 'cnt == {} : '.format(cnt) + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print 'end : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
df_dr_sim.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/10_gold_drug_drug_sim.tsv", sep='\t', index=False)
df_dr_sim.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/10_gold_drug_drug_sim.tsv", sep='\t', index=False)
mat_dr_sim.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/10_gold_drug_drug_sim_mat.tsv", sep='\t')
mat_dr_sim.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/10_gold_drug_drug_sim_mat.tsv", sep='\t')

In [ ]:
disease_mat = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/8_gold_standard_disease_binary_matrix.tsv", index_col=0)

In [ ]:
len(disease_mat.columns)

In [ ]:
#2017.3.13. drug-drug_similarity 구하기
gold_disease_id = disease_mat.columns.values
df_di_sim = pd.DataFrame()
mat_di_sim = pd.DataFrame(columns = gold_disease_id, index=gold_disease_id)
cnt = 0 
print 'start : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for i in gold_disease_id:
    desc1 = disease_mat[i]
    for j in gold_disease_id:
        desc2 = disease_mat[j]
        
        sim = jaccard_similarity_score(desc1, desc2)
        
        #dataframe형
        temp_df = pd.DataFrame(data= [[i, j , sim]], columns=['Disease_id1', 'Disease_id2', 'Similarity'])
        df_di_sim = df_di_sim.append(temp_df)
        # matrix형
        mat_di_sim[i].ix[j] = sim
    cnt = cnt + 1
    if cnt % 100 == 0:
        print 'cnt == {} : '.format(cnt) + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print 'end : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
df_di_sim.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/6_new_disease/10_gold_disease_disease_sim.tsv", sep='\t', index=False)
df_di_sim.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/10_gold_disease_disease_sim.tsv", sep='\t', index=False)
mat_di_sim.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/6_new_disease/10_gold_disease_disease_sim_mat.tsv", sep='\t')
mat_di_sim.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/10_gold_disease_disease_sim_mat.tsv", sep='\t')

In [ ]:
#2017.3.13. drug-disease distance #normalize 한 버전

In [4]:
df_dr_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/10_gold_drug_drug_sim.tsv")
df_di_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/10_gold_disease_disease_sim.tsv")
df_asso = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/7_gold_association.tsv")

In [6]:
gold_drug_id = df_asso['drug_id'].drop_duplicates()
gold_disease_id = df_asso['disease_id'].drop_duplicates()
di_dr_mat = pd.DataFrame(columns = gold_drug_id, index= gold_disease_id) #di에 대한 dr의 distance보기
cnt = 0 
print 'start : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for i in gold_drug_id:
    other_drugs_sim = df_dr_sim[(df_dr_sim.Drug_id1 == i) & (df_dr_sim.Drug_id2 != i)] # 검사할 drug를 제외한 다른 drug들
    for j in gold_disease_id:
        target_disease = df_asso[(df_asso.disease_id ==int(j))] # 검사할 disease
        flags = other_drugs_sim['Drug_id2'].isin(target_disease['drug_id']) # 검사할 disease와 다른 drug들의 association여부
        sum_selected_drug = other_drugs_sim[flags]['Similarity'].sum() # association이 있는 것들의 합
        sum_all_drug = other_drugs_sim['Similarity'].sum() # similarity의 전체 합
        norm = sum_selected_drug / sum_all_drug #normalize한 값
        di_dr_mat[i].ix[j] = norm
    cnt = cnt + 1
    if cnt % 100 == 0:
        print 'cnt == {} : '.format(cnt) + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print 'end : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')

start : 2017-03-15 17:06:06
cnt == 100 : 2017-03-15 17:06:42
cnt == 200 : 2017-03-15 17:07:18
cnt == 300 : 2017-03-15 17:07:53
cnt == 400 : 2017-03-15 17:08:29
end : 2017-03-15 17:08:54


In [7]:
#2017.3.13. disease-drug distance #normalize 한 버전
dr_di_mat = pd.DataFrame(columns = gold_drug_id, index= gold_disease_id) #dr에 대한 di의 distance보기
cnt = 0 
print 'start : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for i in gold_disease_id:
    other_diseases_sim = df_di_sim[(df_di_sim.Disease_id1 == int(i)) & (df_di_sim.Disease_id2 != int(i))] # 검사할 disease를 제외한 다른 disease들
    for j in gold_drug_id:
        target_drug = df_asso[(df_asso.drug_id ==j)] # 검사할 drug
        flags = other_diseases_sim['Disease_id2'].isin(target_drug['disease_id']) # 검사할 drug와 다른 disease들의 association여부
        sum_selected_disease = other_diseases_sim[flags]['Similarity'].sum() # association이 있는 것들의 합
        sum_all_disease = other_diseases_sim['Similarity'].sum() # similarity의 전체 합
        norm = sum_selected_disease / sum_all_disease #normalize한 값
        dr_di_mat[j].ix[i] = norm
    cnt = cnt + 1
    if cnt % 100 == 0:
        print 'cnt == {} : '.format(cnt) + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print 'end : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')

start : 2017-03-15 17:08:54
cnt == 100 : 2017-03-15 17:10:04
cnt == 200 : 2017-03-15 17:11:13
end : 2017-03-15 17:11:37


In [8]:
bef_distance = -(di_dr_mat+dr_di_mat)
distance_mat = pd.DataFrame(columns = bef_distance.columns.values, index = bef_distance.index.values)
cnt = 0 
print 'start : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for i in bef_distance.columns.values:
    for j in bef_distance.index.values:
        a = bef_distance[i].ix[j]
        distance = np.exp(a)
        distance_mat[i].ix[j] = distance
    cnt = cnt + 1
    if cnt % 100 == 0:
        print 'cnt == {} : '.format(cnt) + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print 'end : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')

start : 2017-03-15 17:11:38
cnt == 100 : 2017-03-15 17:11:45
cnt == 200 : 2017-03-15 17:11:52
cnt == 300 : 2017-03-15 17:11:59
cnt == 400 : 2017-03-15 17:12:06
end : 2017-03-15 17:12:11


In [9]:
print di_dr_mat.min().min()
print di_dr_mat.max().max()
print dr_di_mat.min().min()
print dr_di_mat.max().max()
print (di_dr_mat + dr_di_mat).min().min()
print (di_dr_mat + dr_di_mat).max().max()
print bef_distance.min().min()
print bef_distance.max().max()
print np.exp(bef_distance.min().min())
print np.exp(bef_distance.max().max())

0.0
0.161693741082
0.0
0.0812943794645
0.0
0.241334585413
-0.241334585413
-0.0
0.785578739225
1.0


In [ ]:
distance_mat.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/11_1_drug_disease_distance_norm.tsv", sep='\t')
distance_mat.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/7_new_association/11_1_drug_disease_distance_norm.tsv", sep='\t')

In [10]:
#2017.3.13. drug-disease distance #normalize 안 한 버전

In [11]:
di_dr_mat2 = pd.DataFrame(columns = gold_drug_id, index= gold_disease_id) #di에 대한 dr의 distance보기
cnt = 0 
print 'start : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for i in gold_drug_id:
    other_drugs_sim = df_dr_sim[(df_dr_sim.Drug_id1 == i) & (df_dr_sim.Drug_id2 != i)] # 검사할 drug를 제외한 다른 drug들
    for j in gold_disease_id:
        target_disease = df_asso[(df_asso.disease_id ==int(j))] # 검사할 disease
        flags = other_drugs_sim['Drug_id2'].isin(target_disease['drug_id']) # 검사할 disease와 다른 drug들의 association여부
        sum_selected_drug = other_drugs_sim[flags]['Similarity'].sum() # association이 있는 것들의 합
        #sum_all_drug = other_drugs_sim['Similarity'].sum() # similarity의 전체 합
        #norm = sum_selected_drug / sum_all_drug #normalize한 값
        di_dr_mat2[i].ix[j] = sum_selected_drug
    cnt = cnt + 1
    if cnt % 100 == 0:
        print 'cnt == {} : '.format(cnt) + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print 'end : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')

start : 2017-03-15 17:12:11
cnt == 100 : 2017-03-15 17:12:44
cnt == 200 : 2017-03-15 17:13:17
cnt == 300 : 2017-03-15 17:13:50
cnt == 400 : 2017-03-15 17:14:24
end : 2017-03-15 17:14:48


In [12]:
#2017.3.13. disease-drug distance #normalize 한 버전
dr_di_mat2 = pd.DataFrame(columns = gold_drug_id, index= gold_disease_id) #dr에 대한 di의 distance보기
cnt = 0 
print 'start : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for i in gold_disease_id:
    other_diseases_sim = df_di_sim[(df_di_sim.Disease_id1 == int(i)) & (df_di_sim.Disease_id2 != int(i))] # 검사할 disease를 제외한 다른 disease들
    for j in gold_drug_id:
        target_drug = df_asso[(df_asso.drug_id ==j)] # 검사할 drug
        flags = other_diseases_sim['Disease_id2'].isin(target_drug['disease_id']) # 검사할 drug와 다른 disease들의 association여부
        sum_selected_disease = other_diseases_sim[flags]['Similarity'].sum() # association이 있는 것들의 합
        #sum_all_disease = other_diseases_sim['Similarity'].sum() # similarity의 전체 합
        #norm = sum_selected_disease / sum_all_disease #normalize한 값
        dr_di_mat2[j].ix[i] = sum_selected_disease
    cnt = cnt + 1
    if cnt % 100 == 0:
        print 'cnt == {} : '.format(cnt) + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print 'end : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')

start : 2017-03-15 17:14:48
cnt == 100 : 2017-03-15 17:15:53
cnt == 200 : 2017-03-15 17:16:58
end : 2017-03-15 17:17:21


In [13]:
bef_distance2 = -(di_dr_mat2+dr_di_mat2)
distance_mat2 = pd.DataFrame(columns = bef_distance2.columns.values, index = bef_distance2.index.values)
cnt = 0 
print 'start : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for i in bef_distance2.columns.values:
    for j in bef_distance2.index.values:
        a = bef_distance2[i].ix[j]
        distance2 = np.exp(a)
        distance_mat2[i].ix[j] = distance2
    cnt = cnt + 1
    if cnt % 100 == 0:
        print 'cnt == {} : '.format(cnt) + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print 'end : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')

start : 2017-03-15 17:17:21
cnt == 100 : 2017-03-15 17:17:28
cnt == 200 : 2017-03-15 17:17:36
cnt == 300 : 2017-03-15 17:17:43
cnt == 400 : 2017-03-15 17:17:50
end : 2017-03-15 17:17:56


In [14]:
print di_dr_mat2.min().min()
print di_dr_mat2.max().max()
print dr_di_mat2.min().min()
print dr_di_mat2.max().max()
print (di_dr_mat2 + dr_di_mat2).min().min()
print (di_dr_mat2 + dr_di_mat2).max().max()
print bef_distance2.min().min()
print bef_distance2.max().max()
print distance_mat2.min().min()
print distance_mat2.max().max()

0.0
68.710456942
0.0
18.7744958819
0.0
82.7661771026
-82.7661771026
0.0
1.135287866e-36
1.0


In [ ]:
distance_mat2.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/11_2_drug_disease_distance.tsv", sep='\t')
distance_mat2.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/7_new_association/11_2_drug_disease_distance.tsv", sep='\t')

In [15]:
# TN 찾기 2017.3.14.

In [32]:
# 전체 distance matrix에서 거리가 0.1보다 큰 애들을 찾는다. (거리가 먼 애들)
li_tn = list()
for index, row in distance_mat2.iterrows():
    filtered = row[row> 0.1]
    if not filtered.empty:
        li_tn.append([index, filtered.index.tolist()]) 
df_tn = pd.DataFrame()
for i in li_tn:
    for j in i[1]:
        temp_df = pd.DataFrame(data = [[i[0], j]], columns = ['disease_id', 'drug_id'])
        df_tn = df_tn.append(temp_df)

In [47]:
print len(df_tn)
print len(df_tn.drop_duplicates('disease_id'))
print len(df_tn.drop_duplicates('drug_id'))
df_tn.head(1)

9332
82
166


,disease_id,drug_id
0,102400,DB00659


In [34]:
# 구한 TN이 positive associaion과 겹치는게 있는지 체크
duplicated = pd.DataFrame()
for index, row in df_asso.iterrows():
    li_row = row.tolist()
    temp = df_tn[(df_tn.disease_id == li_row[0]) & (df_tn.drug_id == li_row[1])]
    if temp.empty:
        continue
    else:
        duplicated = duplicated.append(temp)

In [66]:
# duplicated 를 제외한 나머지 dataframe 
duplicated['key'] = 0
tt = pd.merge(left = df_tn, right = duplicated, how='left')
df_tn2 = tt[tt.key != 0][['disease_id', 'drug_id']]

In [67]:
df_tn2.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/7_new_association/11_2_2_true_negative_whole.tsv", sep='\t', index=False)
df_tn2.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/11_2_2_true_negative_whole.tsv", sep='\t', index=False)

In [68]:
# di당 거리가 먼 dr을 일정비율로 뽑는다.
li_tn_di = list()
for index, row in distance_mat2.iterrows():
    idx = int(len(row)*0.1)
    filtered = row.sort_values(ascending=False)[:idx]
    li_tn_di.append([index, filtered.index.tolist()])

df_tn_di = pd.DataFrame()
for i in li_tn_di:
    for j in i[1]:
        temp_df = pd.DataFrame(data=[[i[0], j]], columns = ['disease_id', 'drug_id'])
        df_tn_di= df_tn_di.append(temp_df)

In [78]:
print len(df_tn_di)
print len(df_tn_di.drop_duplicates('disease_id'))
print len(df_tn_di.drop_duplicates('drug_id'))
print len(df_tn_di.drop_duplicates())
df_tn_di.head(2)

11045
235
258
11045


,disease_id,drug_id
0,102300,DB01200
0,102300,DB00454


In [79]:
# 구한 TN이 positive associaion과 겹치는게 있는지 체크
duplicated = pd.DataFrame()
for index, row in df_asso.iterrows():
    li_row = row.tolist()
    temp = df_tn_di[(df_tn_di.disease_id == li_row[0]) & (df_tn_di.drug_id == li_row[1])]
    if temp.empty:
        continue
    else:
        duplicated = duplicated.append(temp)

In [82]:
# duplicated 를 제외한 나머지 dataframe 
duplicated['key'] = 0
tt = pd.merge(left = df_tn_di, right = duplicated, how='left')
df_tn_di2 = tt[tt.key != 0][['disease_id', 'drug_id']]

In [84]:
df_tn_di2.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/7_new_association/11_2_3_true_negative_di.tsv", sep='\t', index=False)
df_tn_di2.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/11_2_3_true_negative_di.tsv", sep='\t', index=False)

In [85]:
# dr당 거리가 먼 di을 일정비율로 뽑는다.
li_tn_dr = list()
for column in distance_mat2:
    idx = int(len(distance_mat2[column])*0.1)
    filtered = distance_mat2[column].sort_values(ascending=False)[:idx]
    li_tn_dr.append([column, filtered.index.tolist()])

df_tn_dr = pd.DataFrame()
for i in li_tn_dr:
    for j in i[1]:
        temp_df = pd.DataFrame(data=[[j, i[0]]], columns = ['disease_id', 'drug_id'])
        df_tn_dr= df_tn_dr.append(temp_df)

In [86]:
print len(df_tn_dr)
print len(df_tn_dr.drop_duplicates('disease_id'))
print len(df_tn_dr.drop_duplicates('drug_id'))
df_tn_dr.head()

10833
114
471


,disease_id,drug_id
0,145200,DB00190
0,182000,DB00190
0,246300,DB00190
0,208085,DB00190
0,137800,DB00190


In [87]:
# 구한 TN이 positive associaion과 겹치는게 있는지 체크
duplicated = pd.DataFrame()
for index, row in df_asso.iterrows():
    li_row = row.tolist()
    temp = df_tn_dr[(df_tn_dr.disease_id == li_row[0]) & (df_tn_dr.drug_id == li_row[1])]
    if temp.empty:
        continue
    else:
        duplicated = duplicated.append(temp)

In [89]:
# duplicated 를 제외한 나머지 dataframe 
duplicated['key'] = 0
tt = pd.merge(left = df_tn_dr, right = duplicated, how='left')
df_tn_dr2 = tt[tt.key != 0][['disease_id', 'drug_id']]

In [91]:
df_tn_dr2.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/7_new_association/11_2_4_true_negative_dr.tsv", sep='\t', index=False)
df_tn_dr2.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/11_2_4_true_negative_dr.tsv", sep='\t', index=False)

In [92]:
desc.head(1)

,drug_id,disease_id,association,C0000737,C0000786,C0001824,C0002170,C0002453,C0002622,C0002792,...,387585,388165,55219,55973,65264,80350,84624,84722,91851,406216
0,DB00190,102300,1,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
# negative descriptor 만들기 -whole
df_tn_desc = pd.DataFrame()
for index, row in df_tn2.iterrows():
    temp_df = desc[(desc.drug_id == row['drug_id']) & (desc.disease_id == int(row['disease_id']))]
    if temp_df.empty:
        continue
    else:
        df_tn_desc = df_tn_desc.append(temp_df)

In [94]:
desc_pos = desc[desc.association ==1]
frames = [desc_pos, df_tn_desc]
whole_desc = pd.concat(frames)

In [95]:
print len(whole_desc)
print len(whole_desc.drop_duplicates())

10720
10720


In [96]:
whole_desc.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/6_2_1_tn_whole_descriptor.tsv", sep='\t', index=False)
whole_desc.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/6_2_1_tn_whole_descriptor.tsv", sep='\t', index=False)

In [ ]:
whole_desc.head()

In [97]:
# negative descriptor 만들기 -disease
df_tn_di_desc = pd.DataFrame()
print 'start : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for index, row in df_tn_di2.iterrows():
    temp_df = desc[(desc.drug_id == row['drug_id']) & (desc.disease_id == int(row['disease_id']))]
    if temp_df.empty:
        continue
    else:
        df_tn_di_desc = df_tn_di_desc.append(temp_df)
print 'end : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')

start : 2017-03-15 19:48:43
end : 2017-03-15 19:57:56


In [98]:
#desc_pos = desc[desc.association ==1]
frames = [desc_pos, df_tn_di_desc]
di_desc = pd.concat(frames)

In [104]:
di_desc.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/6_2_2_tn_di_descriptor.tsv", sep='\t', index=False)
di_desc.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/6_2_2_tn_di_descriptor.tsv", sep='\t', index=False)

In [99]:
# negative descriptor 만들기 -drug
df_tn_dr_desc = pd.DataFrame()
print 'start : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for index, row in df_tn_dr2.iterrows():
    temp_df = desc[(desc.drug_id == row['drug_id']) & (desc.disease_id == int(row['disease_id']))]
    if temp_df.empty:
        continue
    else:
        df_tn_dr_desc = df_tn_dr_desc.append(temp_df)
print 'end : ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')

start : 2017-03-15 19:57:56
end : 2017-03-15 20:11:05


In [100]:
print len(df_tn_dr_desc)
print len(df_tn_dr_desc.drop_duplicates('disease_id'))
print len(df_tn_dr_desc.drop_duplicates('drug_id'))
df_tn_dr_desc.head(1)

10651
54
471


,drug_id,disease_id,association,C0000737,C0000786,C0001824,C0002170,C0002453,C0002622,C0002792,...,387585,388165,55219,55973,65264,80350,84624,84722,91851,406216
54,DB00190,145200,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [101]:
print len(df_tn_dr)
print len(df_tn_dr.drop_duplicates('disease_id'))
print len(df_tn_dr.drop_duplicates('drug_id'))
df_tn_dr.head(1)

10833
114
471


,disease_id,drug_id
0,145200,DB00190


In [102]:
#desc_pos = desc[desc.association ==1]
frames = [desc_pos, df_tn_dr_desc]
dr_desc = pd.concat(frames)

In [103]:
dr_desc.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/6_2_3_tn_dr_descriptor.tsv", sep='\t', index=False)
dr_desc.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/6_2_3_tn_dr_descriptor.tsv", sep='\t', index=False)